In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
from input_configuration import base_year

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'

# Trips per Day by Resident
Average number of trips per person by trip type. These values represent total trips divided by population.

In [6]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\person_trips.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_trip['is_rgc'] = 0
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

### County and Region

In [7]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person.sum()['psexpfac']
    df.loc['Region', 'trexpfac'] = df_trip.sum()['trexpfac']
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': purp + ' Trips'}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region', purp + ' Trips'] = df_trip[df_trip['trip_type'] == purp].sum()['trexpfac']
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    df[['Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [8]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.7,3.5,4.2
Kitsap,0.5,3.4,3.9
Pierce,0.5,3.3,3.8
Snohomish,0.6,3.4,3.9
Region,0.6,3.4,4.0


### Regional Growth Centers

In [9]:
df = trips_per_day('is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
Not in RGC,0.6,3.4,4.0
In RGC,0.9,3.7,4.6
Region,0.6,3.4,4.0


In [10]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.5,4.0
Ballard-Interbay,1.0,3.5,4.5
Bellevue,0.9,3.6,4.6
Bothell Canyon Park,0.8,3.2,4.0
Bremerton,0.6,3.5,4.1
Burien,0.7,3.4,4.1
Cascade,0.5,3.3,3.8
Duwamish,0.8,3.5,4.2
Everett,0.6,3.5,4.2


### Regional Geographies

In [11]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.4,3.9
Core,0.6,3.4,4.0
HCT,0.6,3.4,4.0
Metro,0.7,3.6,4.3
Rural,0.5,3.2,3.7
UU,0.5,3.3,3.8
Region,0.6,3.4,4.0


### Equity Geographies

In [12]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']
df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_disability_reg,0.5,3.4,3.9
hh_elderly_reg,1.7,3.4,5.1
hh_english_reg,0.6,3.5,4.1
hh_poverty_50,0.5,3.4,3.9
hh_poverty_reg,0.6,3.4,4.0
hh_racial_50,0.6,3.5,4.1
hh_racial_reg,0.6,3.5,4.1
hh_youth_reg,0.5,3.4,3.9


In [13]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 0].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[0].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']

df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df['Equity Group'] = ['NOT in '+i for i in df['Equity Group']]
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
NOT in hh_disability_reg,0.6,3.5,4.1
NOT in hh_elderly_reg,1.9,3.4,5.3
NOT in hh_english_reg,0.6,3.4,4.0
NOT in hh_poverty_50,0.6,3.4,4.0
NOT in hh_poverty_reg,0.6,3.4,4.1
NOT in hh_racial_50,0.6,3.4,4.0
NOT in hh_racial_reg,0.6,3.4,4.0
NOT in hh_youth_reg,0.7,3.5,4.1


# Miles Driven per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [14]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_vht = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vht.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]
df_vht = df_vht[df_vht['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vht['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

### County

In [15]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,15.8
Kitsap,15.2
Pierce,17.7
Snohomish,19.8
Region,17.0


### Regional Growth Centers

In [16]:
df = vmt_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Miles per Person
Not in RGC,17.6
In RGC,7.1
Region,17.0


In [17]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,12.0
Ballard-Interbay,10.4
Bellevue,7.1
Bothell Canyon Park,20.2
Bremerton,5.3
Burien,13.9
Cascade,17.2
Duwamish,10.8
Everett,9.2


### Regional Geography

In [18]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,21.6
Core,16.2
HCT,17.3
Metro,11.6
Rural,26.0
UU,19.3
Region,17.0


In [19]:
# vmt_per_person(df_vmt, df_person, ['hh_rg_proposed','hh_county'])

_df_vmt = df_vmt.groupby(['hh_rg_proposed','hh_county']).sum()[['travdist_wt']]
_df_person = df_person.groupby(['hh_rg_proposed','hh_county']).sum()[['psexpfac']]

df = _df_vmt.merge(_df_person, left_index=True, right_index=True)

df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog_county.csv')); incr+=1

## Equity Geography

In [20]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
df_in = result_df.copy()

result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
# result_df.index = ['NOT in '+i for i in result_df.index]


df_out = result_df.copy()

df = df_in.merge(df_out, left_index=True, right_index=True)
df.columns = ['Inside Equity Geography',  'Outside Equity Geography']
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
df

,Inside Equity Geography,Outside Equity Geography
source,,
hh_youth_reg,18.6,15.0
hh_elderly_reg,17.5,16.5
hh_english_reg,15.6,17.8
hh_racial_reg,15.1,18.5
hh_racial_50,14.6,17.6
hh_poverty_reg,15.1,18.3
hh_poverty_50,11.6,17.1
hh_disability_reg,16.9,17.0


# Time Driven per Day by Resident
This value represents total vehicle hours by residents divided by population.


In [21]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vht = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vht.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vht['is_rgc'] = 0
df_vht.loc[df_vht['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vht = df_vht[df_vht['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vht['dorp'] == 1)]

def vht_per_person(df_vht, df_person, geog):
    _df_vht = df_vht.groupby(geog).sum()[['travtime_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vht.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Minutes Driven per Person'] = df['travtime_wt']/df['psexpfac']
    
    return df[['Average Minutes Driven per Person']]

## County

In [22]:
df = vht_per_person(df_vht, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vht_county.csv')); incr+=1
df

,Average Minutes Driven per Person
hh_county,
King,38.9
Kitsap,34.4
Pierce,40.3
Snohomish,42.4
Region,39.6


## Regional Growth Centers

In [23]:
df = vht_per_person(df_vht, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Minutes Driven per Person
Not in RGC,40.7
In RGC,21.8
Region,39.6


In [24]:
df = vht_per_person(df_vht, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vht_rgc.csv')); incr+=1
df

,Average Minutes Driven per Person
hh_rgc,
Auburn,28.1
Ballard-Interbay,34.9
Bellevue,19.6
Bothell Canyon Park,43.0
Bremerton,19.5
Burien,32.4
Cascade,34.1
Duwamish,35.2
Everett,22.9


## Regional Geography

In [25]:
df = vht_per_person(df_vht, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vht_reg_geog.csv')); incr+=1
df

,Average Minutes Driven per Person
hh_rg_proposed,
CitiesTowns,43.9
Core,38.1
HCT,40.0
Metro,32.8
Rural,53.4
UU,42.0
Region,39.6


## Equity Geography

In [26]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vht_per_person(df_vht, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
df_in = result_df.copy()

result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vht_per_person(df_vht, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
# result_df.index = ['NOT in '+i for i in result_df.index]


df_out = result_df.copy()

df = df_in.merge(df_out, left_index=True, right_index=True)
df.columns = ['Inside Equity Geography',  'Outside Equity Geography']
df.to_csv(os.path.join(output_dir,str(incr)+'_vht_equity.csv')); incr+=1
df

,Inside Equity Geography,Outside Equity Geography
source,,
hh_youth_reg,41.5,37.3
hh_elderly_reg,40.8,38.5
hh_english_reg,37.4,40.9
hh_racial_reg,36.4,42.2
hh_racial_50,36.0,40.5
hh_poverty_reg,36.2,42.0
hh_poverty_50,29.6,39.8
hh_disability_reg,38.7,40.3


# Miles Walking and Biking per Day by Resident

These numbers represent total miles spent biking and walking divided by population. These do *not* represent the average trip distances walking or biking. 

County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [27]:
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1


def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

### County

In [28]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.5,0.3,0.8
Kitsap,0.4,0.2,0.7
Pierce,0.4,0.3,0.7
Snohomish,0.4,0.3,0.7
Region,0.5,0.3,0.8


### Regional Growth Center 

In [29]:
df = walk_bike_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'Inside RGC', 'Region']
df

,Walk,Bike,Walk and Bike
Not in RGC,0.4,0.3,0.7
Inside RGC,1.1,0.3,1.4
Region,0.5,0.3,0.8


In [30]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.6,0.4,1.0
Ballard-Interbay,0.7,0.5,1.3
Bellevue,1.0,0.3,1.2
Bothell Canyon Park,0.4,0.5,1.0
Bremerton,0.8,0.3,1.1
Burien,0.5,0.4,0.9
Cascade,0.4,0.2,0.7
Duwamish,0.8,0.2,1.0
Everett,0.8,0.3,1.1


### Regional Geography

In [31]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.4,0.2,0.6
Core,0.4,0.3,0.7
HCT,0.4,0.3,0.7
Metro,0.7,0.3,1.0
Rural,0.3,0.3,0.6
UU,0.4,0.2,0.6
Region,0.5,0.3,0.8


### Equity Geographies

In [32]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
source,,,
hh_youth_reg,0.3,0.4,0.7
hh_elderly_reg,0.3,0.4,0.7
hh_english_reg,0.3,0.5,0.8
hh_racial_reg,0.3,0.5,0.8
hh_racial_50,0.3,0.5,0.8
hh_poverty_reg,0.3,0.5,0.8
hh_poverty_50,0.3,0.6,0.9
hh_disability_reg,0.3,0.5,0.7


In [33]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.index = ['NOT in '+i for i in result_df.index]
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity_inverse.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
NOT in hh_youth_reg,0.3,0.6,0.9
NOT in hh_elderly_reg,0.3,0.5,0.8
NOT in hh_english_reg,0.3,0.5,0.7
NOT in hh_racial_reg,0.3,0.4,0.7
NOT in hh_racial_50,0.3,0.5,0.7
NOT in hh_poverty_reg,0.3,0.5,0.7
NOT in hh_poverty_50,0.3,0.5,0.8
NOT in hh_disability_reg,0.3,0.5,0.8


# Time Walking and Biking per Day by Resident

These numbers represent total times spent biking and walking divided by population. These do *not* represent the average trip times walking or biking. 

County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

### County

In [34]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vht = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vht.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vht['is_rgc'] = 0
df_vht.loc[df_vht['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1



def walk_bike_per_person_time(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travtime_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travtime_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

df = walk_bike_per_person_time(df_vht, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,10.8,2.0,12.8
Kitsap,8.8,1.5,10.3
Pierce,8.2,1.8,10.0
Snohomish,7.9,1.7,9.6
Region,9.6,1.9,11.4


### Regional Growth Center 

In [35]:
df = walk_bike_per_person_time(df_vht, df_person, 'is_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_rgc_binary.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
is_rgc,,,
0,8.8,1.8,10.6
1,22.3,2.2,24.5
Region,9.6,1.9,11.4


In [36]:
df = walk_bike_per_person_time(df_vht, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_rgc.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,12.2,2.7,14.9
Ballard-Interbay,14.9,3.5,18.4
Bellevue,19.5,1.7,21.3
Bothell Canyon Park,8.6,3.6,12.2
Bremerton,16.2,1.7,17.9
Burien,9.3,2.8,12.1
Cascade,8.7,1.5,10.3
Duwamish,16.2,1.1,17.3
Everett,15.2,2.3,17.5


### Regional Geography

In [37]:
df = walk_bike_per_person_time(df_vht, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_rg.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,7.5,1.6,9.1
Core,8.8,1.9,10.7
HCT,8.2,1.7,10.0
Metro,14.0,2.0,16.0
Rural,5.1,2.1,7.2
UU,7.2,1.6,8.9
Region,9.6,1.9,11.4


### Equity Geography

In [38]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person_time(df_vht, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
source,,,
hh_youth_reg,1.8,8.0,9.8
hh_elderly_reg,1.9,8.9,10.8
hh_english_reg,1.9,9.7,11.6
hh_racial_reg,1.9,10.3,12.2
hh_racial_50,2.0,10.1,12.1
hh_poverty_reg,2.0,10.3,12.2
hh_poverty_50,2.3,11.8,14.2
hh_disability_reg,1.9,9.4,11.3


In [39]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person_time(df_vht, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.index = ['NOT in '+i for i in result_df.index]
result_df.to_csv(os.path.join(output_dir,str(incr)+'_minutes_bikewalk_equity_inverse.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
NOT in hh_youth_reg,2.0,11.4,13.4
NOT in hh_elderly_reg,1.9,10.1,12.0
NOT in hh_english_reg,1.8,9.5,11.3
NOT in hh_racial_reg,1.8,9.0,10.8
NOT in hh_racial_50,1.8,9.4,11.3
NOT in hh_poverty_reg,1.8,9.1,10.9
NOT in hh_poverty_50,1.9,9.5,11.4
NOT in hh_disability_reg,1.9,9.7,11.6
